In [1]:
import pickle
from multiprocessing.pool import Pool

import numpy as np
import pandas as pd

from countries import get_countries

with open("data/actors.pkl", "rb") as f:
    df = pd.DataFrame(sorted(pickle.load(f)), columns=["nconst"])

def parallelize_dataframe(df, func, n_cores=12):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [2]:
df = parallelize_dataframe(df, get_countries)

In [3]:
df.to_pickle("data/birth_countries.pkl")
df

,nconst,birth_country
0,nm0000001,USA
1,nm0000002,USA
2,nm0000003,France
3,nm0000006,Sweden
4,nm0000007,USA
...,...,...
13414,nm9656000,
13415,nm9706424,
13416,nm9712684,
13417,nm9845146,India
